In [1]:
import seagul.envs
import ray
import ray.rllib.agents.ars as ars
from ray.tune.logger import pretty_print
import tensorflow as tf


from ray.tune.registry import register_env
from seagul.envs.mujoco.five_link import FiveLinkWalkerEnv

def env_creator(env_config):
    return FiveLinkWalkerEnv()  # return an env instance

register_env("five_link-v3", env_creator)

ray.init()
config = ars.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
config["eager"] = False
config["model"]["fcnet_hiddens"] = []

trainer = ars.ARSTrainer(config=config, env="five_link-v3")

# Can optionally call trainer.restore(path) to load a checkpoint.

for i in range(1):
   # Perform one iteration of training the policy with ARS
    result = trainer.train()
    print(pretty_print(result))

    #if i % 100 == 0:
    checkpoint = trainer.save()
    print("checkpoint saved at", checkpoint)

/Users/sgillen/anaconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/sgillen/anaconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/sgillen/anaconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/sgillen/anaconda3/envs/ray/lib/python3.6/s

(pid=26172) /Users/sgillen/anaconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=26172)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=26172) /Users/sgillen/anaconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=26172)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=26172) /Users/sgillen/anaconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=26172)   _np_qint16 = np.dtype([("q

In [2]:
import numpy as np
trainer.policy.compute(np.random.randn(13))

trainer.policy

In [1]:
import seagul.envs
import ray
import ray.rllib.agents.ars as ars
from ray.tune.logger import pretty_print
import tensorflow as tf


from ray.tune.registry import register_env
from seagul.envs.mujoco.five_link import FiveLinkWalkerEnv

def env_creator(env_config):
    return FiveLinkWalkerEnv()  # return an env instance

register_env("five_link-v3", env_creator)

ray.init()
config = ars.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
config["eager"] = False
trainer = ars.ARSTrainer(config=config, env="five_link-v3")

trainer.load("/Users/sgillen/ray_results/ARS_five_link-v3_2019-10-21_14-02-55lwgccrp3/checkpoint_10/checkpoint-10")

/Users/sgillen/anaconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/sgillen/anaconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/sgillen/anaconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/sgillen/anaconda3/envs/ray/lib/python3.6/s

AttributeError: 'ARSTrainer' object has no attribute 'load'

(pid=15760) /Users/sgillen/anaconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=15760)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=15760) /Users/sgillen/anaconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=15760)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=15760) /Users/sgillen/anaconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=15760)   _np_qint16 = np.dtype([("q

In [4]:
trainer.restore("/Users/sgillen/ray_results/ARS_five_link-v3_2019-10-21_14-02-55lwgccrp3/checkpoint_10/checkpoint-10")
trainer.policy.compute

2019-10-21 14:06:17,881	INFO trainable.py:358 -- Restored from checkpoint: /Users/sgillen/ray_results/ARS_five_link-v3_2019-10-21_14-02-55lwgccrp3/checkpoint_10/checkpoint-10
2019-10-21 14:06:17,884	INFO trainable.py:365 -- Current state after restoring: {'_iteration': 10, '_timesteps_total': 18389, '_time_total': 28.655691623687744, '_episodes_total': None}


<bound method GenericPolicy.compute of <ray.rllib.agents.ars.policies.GenericPolicy object at 0x141a09940>>

In [58]:
trainer.policy
print(trainer.policy.num_params)

70404


In [60]:
trainer.policy.get_weights()

array([-0.05028875,  0.29160112, -0.051689  , ..., -0.00064282,
        0.00532838,  0.00534475], dtype=float32)

In [61]:
trainer.config

{'monitor': False,
 'log_level': 'INFO',
 'callbacks': {'on_episode_start': None,
  'on_episode_step': None,
  'on_episode_end': None,
  'on_sample_end': None,
  'on_train_result': None,
  'on_postprocess_traj': None},
 'ignore_worker_failures': False,
 'log_sys_usage': True,
 'eager': False,
 'eager_tracing': False,
 'no_eager_on_workers': False,
 'model': {'conv_filters': None,
  'conv_activation': 'relu',
  'fcnet_activation': 'tanh',
  'fcnet_hiddens': [256, 256],
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': True,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action_reward': False,
  'state_shape': None,
  'framestack': True,
  'dim': 84,
  'grayscale': False,
  'zero_mean': True,
  'custom_preprocessor': None,
  'custom_model': None,
  'custom_action_dist': None,
  'custom_options': {}},
 'optimizer': {},
 'gamma': 0.99,
 'horizon': None,
 'soft_horizon': False,
 'no_done_at_end': False,
 'env_config': {},
 'env': 